In [1]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from data import river_training_samples
from data import non_river_training_samples

In [2]:
Im, Im1, Im2, Im3 = Image.open('Images/1.gif', 'r'), Image.open('Images/2.gif', 'r'), Image.open('Images/3.gif', 'r'), Image.open('Images/4.gif', 'r')
tmp, tmp1, tmp2, tmp3 = list(Im.getdata()), list(Im1.getdata()), list(Im2.getdata()), list(Im3.getdata())
tmp = np.array(tmp).reshape(512, 512)
tmp1 = np.array(tmp1).reshape(512, 512)
tmp2 = np.array(tmp2).reshape(512, 512)
tmp3 = np.array(tmp3).reshape(512, 512)
All_data = []
for i in range(512) :
    Tmp = []
    for j in range(512) :
        Tmp1 = []
        Tmp1.append(tmp[i][j])
        Tmp1.append(tmp1[i][j])
        Tmp1.append(tmp2[i][j])
        Tmp1.append(tmp3[i][j])
        Tmp1 = np.array(Tmp1)
        Tmp.append(Tmp1)
    Tmp = np.array(Tmp)
    All_data.append(Tmp)
All_data = np.array(All_data)

In [3]:
data_training = np.vstack((river_training_samples, non_river_training_samples))

In [4]:
Mean_0 = np.mean(non_river_training_samples, axis=0)
Val0 = np.zeros(non_river_training_samples.shape)
for i in range(non_river_training_samples.shape[0]) :
    for j in range(non_river_training_samples.shape[1]) :
        Val0[i][j] = non_river_training_samples[i][j] - Mean_0[j]
cov_0 = (float(1) / float(non_river_training_samples.shape[0])) * np.dot(np.transpose(Val0), Val0)

In [5]:
Mean_1 = np.mean(river_training_samples, axis=0)
Val1 = np.zeros(river_training_samples.shape)
for i in range(river_training_samples.shape[0]) :
    for j in range(river_training_samples.shape[1]) :
        Val1[i][j] = river_training_samples[i][j] - Mean_1[j]
cov_1 = (float(1) / float(river_training_samples.shape[0])) * np.dot(np.transpose(Val1), Val1)

In [6]:
Image_final = np.zeros((512, 512))

In [7]:
for i in range(512) :
    for j in range(512) :
        data_modified = np.zeros(All_data[i][j].shape)
        for k in range(data_modified.shape[0]) :
            data_modified[k] = All_data[i][j][k] - Mean_1[k]
        class_river = np.dot(np.dot(np.transpose((data_modified)), np.linalg.inv(cov_1)), data_modified)
        prediction_river = (float(1) / ((float(2) * np.pi) ** 2)) * (float(1) / (np.sqrt(np.linalg.det(cov_1)))) * np.exp((float(-0.5)) * class_river)
        data_modified = np.zeros(All_data[i][j].shape)
        for k in range(data_modified.shape[0]) :
            data_modified[k] = All_data[i][j][k] - Mean_0[k]
        class_non_river = np.dot(np.dot(np.transpose((data_modified)), np.linalg.inv(cov_0)), data_modified)
        prediction_non_river =  (float(1) / ((float(2) * np.pi) ** 2)) * (float(1) / (np.sqrt(np.linalg.det(cov_0)))) * np.exp((float(-0.5)) * class_non_river)
        P1 = 0.7
        P2 = 0.3
        if(P1 * prediction_river >= P2 * prediction_non_river) :
            Image_final[i][j] = 0
        else :
            Image_final[i][j] = 255

In [8]:
Im = Image.fromarray(Image_final)
Im.show()

In [9]:
Im1 = Image.fromarray(255 - Image_final)
Im1.show()